# CIS 399 Final Project - Steve Kent


Project Goals:
I am interested in creating new sentences based on the top POS (parts of speech) words used in gothic literature. The tool I will make can take a gothic sentence and create a new sentence with the same POS relations but new words that are commonly found in gothic books. This tool would ultimately be used to create new books. Creating a book would obviously be much more challenging but this is a start in that direction. 

### Part 1: Getting top POS words

First I will go through our gothic books and pull out the most commonly used words for each POS. 

The next 3 cells are used for setting up the necessary components. Includes downloading the gothic table and nltk things. Also, I prepare a POS dictionary and setup a sentence wrangler. 

In [332]:
import pandas as pd

gothic_table = pd.read_csv('https://docs.google.com/spreadsheets/d/e/2PACX-1vQqRwyE0ceZREKqhuaOw8uQguTG6Alr5kocggvAnczrWaimXE8ncR--GC0o_PyVDlb-R6Z60v-XaWm9/pub?output=csv',
                          encoding='utf-8')
import nltk
from nltk.tree import Tree
nltk.download('averaged_perceptron_tagger')
nltk.download('maxent_ne_chunker')
nltk.download('words')

import string

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /Users/stevekent/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package maxent_ne_chunker to
[nltk_data]     /Users/stevekent/nltk_data...
[nltk_data]   Package maxent_ne_chunker is already up-to-date!
[nltk_data] Downloading package words to /Users/stevekent/nltk_data...
[nltk_data]   Package words is already up-to-date!


In [333]:
posDict = {}
posDict["CC"] = {} 
posDict["CD"] = {}
posDict["DT"] = {}
posDict["EX"] = {}
posDict["FW"] = {}
posDict["IN"] = {}
posDict["JJ"] = {}
posDict["JJR"] = {}
posDict["JJS"] = {}
posDict["LS"] = {}
posDict["MD"] = {}
posDict["NN"] = {}
posDict["NNS"] = {}
posDict["NNP"] = {}
posDict["NNPS"] = {}
posDict["PDT"] = {}
posDict["POS"] = {}
posDict["PRP"] = {}
posDict["PRP$"] = {}
posDict["RB"] = {}
posDict["RBR"] = {}
posDict["RBS"] = {}
posDict["RP"] = {}
posDict["SYM"] = {}
posDict["TO"] = {}
posDict["UH"] = {}
posDict["VB"] = {}
posDict["VBD"] = {}
posDict["VBG"] = {}
posDict["VBN"] = {}
posDict["VBP"] = {}
posDict["VBZ"] = {}
posDict["WDT"] = {}
posDict["WP"] = {}
posDict["WP$"] = {}
posDict["WRB"] = {}


In [334]:
from nltk.tokenize import WordPunctTokenizer
word_punct_tokenizer = WordPunctTokenizer()


legals = 'abcdefghijklmnopqrstuvwxyz'

def sentence_wrangler(sentence, legal_chars):
    tokes = word_punct_tokenizer.tokenize(sentence)
    result = []
    removed = []
    check = 0
    for word in tokes:
        word = word.lower()
        for i in word:
            if i in legal_chars:
                check = 1
            else: 
                check = 0
                break
        if check == 1:
            result.append(word.lower())
        else:
            removed.append(word.lower())

    return (result, removed)


Here I am going to go throught the gothic books and create the POS dictionary. First I wrangle the sentence to get words without puncuation or capital letters. Then I get the POS tag for the words in the sentence. Finally, I go through the output from pos_tag and add items to the POS dictionary.

In [337]:
count = 2797 #used for timing
for j in range(len(gothic_table)):
    sentence = gothic_table.loc[j, 'text']
    author = gothic_table.loc[j, 'author']
    words = sentence_wrangler(sentence, legals)[0]
    pos = nltk.pos_tag(words)
    for i in range(len(pos)):
        if pos[i][1] in posDict:
            if pos[i][0] not in posDict[pos[i][1]]:
                if author == "EAP":
                    posDict[pos[i][1]][pos[i][0]] = [1, 0, 0]
                if author == "HPL":
                    posDict[pos[i][1]][pos[i][0]] = [0, 1, 0]
                if author == "MWS":
                    posDict[pos[i][1]][pos[i][0]] = [0, 0, 1]
            else:
                if author == "EAP":
                    posDict[pos[i][1]][pos[i][0]][0] += 1
                if author == "HPL":
                    posDict[pos[i][1]][pos[i][0]][1] += 1
                if author == "MWS":
                    posDict[pos[i][1]][pos[i][0]][2] += 1
    #timing
    if j == 0:
        print "starting..."
    if j == count:
        print ('%d/19579 finished...' % count)
        count += 2797
        if count == 19579:
            print ('%d/19579 finished...' % count)
            print "done!"

starting...
2797/19579 finished...
5594/19579 finished...
8391/19579 finished...
11188/19579 finished...
13985/19579 finished...
16782/19579 finished...
19579/19579 finished...
done!


Next I will create a function called "most_important" that takes an author, a POS and the POS dictionary and returns the most important words for the author for that POS. This function will use the calculate_importance function from midterm 1 to determine which author claims each word most important to them. 

In [338]:
import math

def calculate_importance(counts_list):
    count = 0.0
    dont_use_word = 0.0
    results = []
    for i in counts_list:
        count += 1
        if i == 0:
            dont_use_word += 1
    for i in counts_list:
        tf = i
        idf = math.log10(count/(count-dont_use_word))
        results.append(tf*idf)
    return results

In [339]:
def most_important(relation, author, dictionary):
    output = []
    for word in dictionary[relation].items():
        result = calculate_importance(word[1])
        if author == "EAP":
            if result[0] > result[2] and result[0] > result[1]:
                output.append(word[0])
        if author == "HPL":
            if result[1] > result[2] and result[1] > result[0]:
                output.append(word[0])
        if author == "MWS":
            if result[2] > result[0] and result[2] > result[1]:
                output.append(word[0])
    return output
            

In [340]:
out = most_important('VBD', 'HPL', posDict)
print out

[u'hath', u'saved', u'crooned', u'sputtered', u'patched', u'characterised', u'slippered', u'wooded', u'sunk', u'festered', u'favoured', u'beaten', u'pawed', u'sidled', u'stared', u'ploughed', u'represented', u'shriek', u'buckled', u'proportioned', u'oozed', u'gloated', u'postponed', u'reckoned', u'charred', u'uncased', u'menaced', u'hole', u'moulded', u'give', u'flung', u'poked', u'kranon', u'plied', u'infested', u'dreamed', u'rewarded', u'fit', u'diseased', u'dovetailed', u'disembodied', u'swilled', u'towered', u'laboured', u'mangled', u'elwood', u'hanged', u'waddled', u'haired', u'washed', u'disputed', u'conflicted', u'eared', u'ritual', u'delved', u'chopped', u'twitched', u'dwelt', u'masted', u'courted', u'top', u'twisted', u'blamed', u'stuffed', u'excelled', u'injected', u'lunged', u'garlanded', u'mercy', u'matted', u'scraped', u'soaked', u'cleft', u'degenerated', u'glimpsed', u'reverberated', u'circled', u'whar', u'episode', u'inferred', u'enshrined', u'alight', u'flouted', u'larn

Now I created a function that will go through the POS dict and find the most common words for all authors instead of just one. Also, I demonstrate how this could be used using the random.choice function on the returned list. This will give me a random top word to be used in creating a new gothic sentence. This function will not be used further but is an alternate route to picking out words. 

In [341]:
import collections as c 
import random as r

def getMostCommon(relation, sourceDict): 
    for subdict in sourceDict.items(): 
        if subdict[0] == relation: 
            out = c.Counter(subdict[1]) 
            return out.most_common()[:50]
        
result = getMostCommon('JJ', posDict)
print (r.choice(result))

(u'small', [199, 289, 135])


### Part 2: Creating Partial Sentence Tool

This tool will act sort of like a Mad Libs machine for gothic literature. I am going to give user option to take either a random word from what I've done above or take a random synonym. The tool must also accept partially completed sentences so first I will need to update sentence wrangler so that it can handle partially completed sentences. 

Sentences that are incomplete will need to follow this general format:

"Victor Frankenstein VBZ the creature in his laboratory"

The incomplete words will need to have their POS as a place holder. This also allows for the entire sentence to be wildcards. 

In [342]:
test_sentence = "Victor Frankenstein builds the NN in his laboratory"

legals2 = 'abcdefghijklmnopqrstuvwxyzABCDEFGHIJKLMNOPQRSTUVWXYZ'

#Similar to original sentence wrangler, just ignores capitalization.
def sentence_wrangler2(sentence, legal_chars):
    tokes = word_punct_tokenizer.tokenize(sentence)
    result = []
    removed = []
    check = 0
    for word in tokes:
        for i in word:
            if i in legal_chars:
                check = 1
            else: 
                check = 0
                break
        if check == 1:
            result.append(word)
        else:
            removed.append(word)

    return (result, removed)
            
words = sentence_wrangler2(test_sentence, legals2)[0]
print words

['Victor', 'Frankenstein', 'builds', 'the', 'NN', 'in', 'his', 'laboratory']


Now on to the actual tool. Note that each time the function is called, a new word will be chosen and the sentence will be different. This tool isn't perfect because some words don't work in the original sentence but it is pretty interesting and at times amusing.  

In [343]:
def partial_sentence(sentence, author):
    words = sentence_wrangler2(sentence, legals2)[0]
    new_sentence = []
    return_string = ""
    for word in words:
        if word in posDict:
            most = most_important(word, author, posDict) #using most_important
            replace = r.choice(most)
            new_sentence.append(replace)
        else:
            new_sentence.append(word)
    for word in new_sentence:
        return_string += word + " "
    return return_string

In [346]:
print partial_sentence(test_sentence, 'MWS')

Victor Frankenstein builds the mime in his laboratory 


### Part 3: Complete Sentence to New Sentence Tool

The final tool takes a sentence, extracts the POS relations and building a new sentence. The new sentence can be built with either gothic words or synonyms for each word. This tool cannot accept wildcards and will need to be a complete sentence. The gothic flavor argument is where the user gets the option between a gothic word or a synonym (True for "yes gothic", False for "no, I want synonyms"). The POStoChange argument will allow user to change only one type of POS in the sentence (for example, change just the nouns). If POStoChange is set to an empty string/is unspecified, then the function will change all words. 

The following two cells are helper functions. Get Syns returns a list of synonym words. Show POS simply outputs the POS for each word in a sentence so the user can choose which POS they want to change.

In [278]:
from nltk.corpus import wordnet
def get_syns(word):
    synonyms = []
    for syn in wordnet.synsets(word):
        for lem in syn.lemmas():
            synonyms.append(lem.name())
    return list(set(synonyms))

In [305]:
def show_pos(sentence):
    words = sentence_wrangler2(sentence, legals2)[0]
    pos = nltk.pos_tag(words)
    print pos

In [350]:
test_sentence2 = "Victor Frankenstein builds the creature in his laboratory"
test_sentence3 = "the monster kills Frankenstein's best friend Henry Clerva."

def completely_new_sentence(sentence, gothic_flavor, POStoChange):
    words = sentence_wrangler2(sentence, legals2)[0]
    pos = nltk.pos_tag(words)
    new_sentence = []
    return_string = ""
    if gothic_flavor: 
        for tup in pos:
            if len(POStoChange) > 1:
                if tup[1] == POStoChange:
                    most = most_important(tup[1], author, posDict) #using most_important
                    replace = r.choice(most)
                    new_sentence.append(replace)
                else:
                    new_sentence.append(tup[0])
            else:
                most = most_important(tup[1], author, posDict) #using most_important
                replace = r.choice(most)
                new_sentence.append(replace)
    else:
        for tup in pos:
            if len(POStoChange) > 1:
                if tup[1] == POStoChange:
                    syn = get_syns(tup[0])
                    if syn:
                        replace = r.choice(syn)
                        new_sentence.append(replace)
                    else:
                        new_sentence.append(tup[0])
                else:
                    new_sentence.append(tup[0])
            else:
                most = most_important(tup[1], author, posDict) #using most_important
                replace = r.choice(most)
                new_sentence.append(replace)
    for word in new_sentence:
        return_string += word + " "
    return return_string

In [353]:
show_pos(test_sentence2)
print "====================================================="
print completely_new_sentence(test_sentence2, False, "")

[('Victor', 'NNP'), ('Frankenstein', 'NNP'), ('builds', 'VBZ'), ('the', 'DT'), ('creature', 'NN'), ('in', 'IN'), ('his', 'PRP$'), ('laboratory', 'NN')]
xiith yog enjoys azathoth greaser arkham hiram postumius 


### Conclusion

I think these two tools are pretty interesting and act as an effective MadLib-type machine. Adding specifc words from specific authors is a good step towards writing a new book in the sytle of one author. That would be the ultimate goal for these tools even if that goal is impossible. I think the next logical step would be to look at commonly used POS patterns by certain authors and then creating those kinds of sentences. 